# Modelo MLP para Previsão de Preços de Criptomoedas

## Importação das Bibliotecas

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
import joblib

## PASSO 1: Carregar e Preparar os Dados

**Objetivo:** Ler o arquivo CSV e garantir que ele esteja em ordem.

In [ ]:
# Carrega os dados do arquivo CSV
df = pd.read_csv(r"C:\Users\david\Desktop\trading-algoritmico\data\fechamentos\SOLUSDT_1h_data.csv")

# Garante que a coluna 'timestamp' seja do tipo data e ordena os dados
df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.sort_values('timestamp')
df = df.reset_index(drop=True)

# Remove colunas com nomes "Unnamed".
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

print(f"Dados carregados: {df.shape[0]} registros.")
print(df.head())

## PASSO 2: Preparar os Dados para o Modelo

**Objetivo:** Transformar os dados brutos em um formato que a rede neural entenda (features e target).

In [1]:
# --- Definição das Variáveis ---
FEATURE_COLUMNS = ['open', 'high', 'low', 'close', 'volume']
TARGET_COLUMN = 'close'

# SEQUENCE_LENGTH: Quantos períodos olharemos para trás para prever o próximo.
# Por exemplo, com 24, usaremos os dados das últimas 24 horas para fazer uma previsão.
SEQUENCE_LENGTH = 24

# --- Criação das Features (X) e do Alvo (y) ---
# O alvo (y) será binário: 1 se o preço subir, 0 se não.
# `df[TARGET_COLUMN].shift(-1)` pega o preço de fechamento do *próximo* período.
y = (df[TARGET_COLUMN].shift(-1) > df[TARGET_COLUMN]).astype(int)

# As features (X) serão os dados OHLCV.
X_raw = df[FEATURE_COLUMNS]

print(f"Features definidas: {FEATURE_COLUMNS}")
print(f"Sequence length: {SEQUENCE_LENGTH}")
print(f"Shape dos dados brutos: {X_raw.shape}")

NameError: name 'df' is not defined

In [ ]:
# --- Normalização dos Dados ---
# Importante para redes neurais, pois ajuda na convergência do modelo
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_raw)

print(f"Dados normalizados. Shape: {X_scaled.shape}")
print(f"Min: {X_scaled.min()}, Max: {X_scaled.max()}")

In [ ]:
# --- Criação das Sequências ---
# Cria sequências de dados para o modelo
X_sequences, y_sequences = [], []
for i in range(len(X_scaled) - SEQUENCE_LENGTH):
    X_sequences.append(X_scaled[i : i + SEQUENCE_LENGTH])
    y_sequences.append(y[i + SEQUENCE_LENGTH -1]) # O alvo corresponde ao final da sequência

X = np.array(X_sequences)
y = np.array(y_sequences)

print(f"Sequências criadas. X shape: {X.shape}, y shape: {y.shape}")

In [ ]:
# --- Reshape do X para o MLP ---
# O MLP espera uma entrada 2D, então precisa achatar as sequências
num_samples = X.shape[0]
num_features_flat = X.shape[1] * X.shape[2]
X = X.reshape(num_samples, num_features_flat)

print(f"Shape de X (features): {X.shape}")
print(f"Shape de y (alvo): {y.shape}")

In [ ]:
# --- Divisão em Treino e Teste ---
# Divisão dos dados em treino e teste
split_ratio = 0.8
split_index = int(len(X) * split_ratio)

X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

print(f"Amostras de treino: {len(X_train)}")
print(f"Amostras de teste: {len(X_test)}")
print(f"Distribuição do target no treino: {np.bincount(y_train)}")
print(f"Distribuição do target no teste: {np.bincount(y_test)}")

## PASSO 3: Construir o Modelo (MLP)

**Objetivo:** Definir a arquitetura da nossa rede neural.

In [ ]:
model = Sequential()

# Camada de Entrada: Define o formato da nossa entrada
model.add(Input(shape=(num_features_flat,)))

# 1ª Camada Oculta: Primeira camada que aprende os padrões
model.add(Dense(units=64, activation='relu'))

# Camada de Dropout: Ajuda a prevenir overfitting
model.add(Dropout(rate=0.2))

# 2ª Camada Oculta: Segunda camada para aprender padrões mais complexos
model.add(Dense(units=32, activation='relu'))

# Camada de Saída: Gera a previsão final
model.add(Dense(units=1, activation='sigmoid'))

NameError: name 'Sequential' is not defined

In [ ]:
# --- Compilação do Modelo ---
# O modelo é compilado com o otimizador Adam e a função de perda binary_crossentropy, adequada para problemas de classificação binária
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Mostra um resumo da arquitetura do modelo
model.summary()

## PASSO 4: Treinar o Modelo

**Objetivo:** Alimentar o modelo com os dados de treino para que ele aprenda.

In [ ]:
history = model.fit(
    X_train,
    y_train,
    epochs=20,  # epochs: Quantas vezes o modelo verá todo o conjunto de dados de treino.
    batch_size=32,  # batch_size: Quantas amostras o modelo vê antes de atualizar seus pesos.
    validation_data=(X_test, y_test),  # Dados para validar o modelo a cada época.
    verbose=1 # Mostra uma barra de progresso.
)

## PASSO 5: Avaliar o Modelo

**Objetivo:** Verificar o quão bem o modelo se saiu nos dados de teste.

In [ ]:
print("\n--- PASSO 5: Avaliando o Modelo ---")

# Fazer previsões nos dados de teste. A saída será uma probabilidade
probabilities = model.predict(X_test)

# Converter probabilidades em classes (0 ou 1) usando um limiar de 0.5
predictions = (probabilities > 0.5).astype(int)

# Calcular e mostrar as métricas
accuracy = accuracy_score(y_test, predictions)
cm = confusion_matrix(y_test, predictions)
report = classification_report(y_test, predictions, target_names=['Não Sobe', 'Sobe'])

print(f"Acurácia no Teste: {accuracy * 100:.2f}%")
print("\nRelatório de Classificação:")
print(report)

In [ ]:
print("\nMatriz de Confusão:")
# Um mapa de calor para visualizar a matriz de confusão
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Não Sobe', 'Sobe'], yticklabels=['Não Sobe', 'Sobe'])
plt.xlabel('Predito')
plt.ylabel('Real')
plt.title('Matriz de Confusão')
plt.show()

## PASSO 6: Visualizar o Treinamento

**Objetivo:** Plotar gráficos para ver se o modelo aprendeu bem ou se teve overfitting.

In [ ]:
plt.figure(figsize=(12, 5))

# Gráfico da Acurácia
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Acurácia de Treino')
plt.plot(history.history['val_accuracy'], label='Acurácia de Validação')
plt.title('Acurácia ao Longo das Épocas')
plt.xlabel('Época')
plt.ylabel('Acurácia')
plt.legend()

# Gráfico da Perda (Loss)
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Perda de Treino')
plt.plot(history.history['val_loss'], label='Perda de Validação')
plt.title('Perda ao Longo das Épocas')
plt.xlabel('Época')
plt.ylabel('Perda')
plt.legend()

plt.tight_layout()
plt.show()

## PASSO 7: Salvar o Modelo

**Objetivo:** Salvar o modelo treinado e o normalizador para uso futuro.

In [ ]:
# Por enquanto não é necessário salvar o modelo
MODEL_PATH = "mlp_basic_model.h5"
SCALER_PATH = "basic_scaler.pkl"

model.save(MODEL_PATH)
joblib.dump(scaler, SCALER_PATH)

print(f"Modelo salvo em: {MODEL_PATH}")
print(f"Normalizador salvo em: {SCALER_PATH}")

## Conclusão

### Já integrado:

1. **Carrega dados históricos**
2. **Prepara os dados**
3. **Constrói um MLP**
4. **Treina o modelo**
5. **Avalia o desempenho**
6. **Visualiza o treinamento**
7. **Salva o modelo**

### Próximos Passos:
- ENTENDER POR QUE O MODELO SÓ FALA QUE VAI SUBIR
- Adicionar features técnicas, prepara-las e dar ao treino do modelo
- Implementar validação cruzada temporal
- Testar diferentes janelas temporais